In [ ]:
# default_exp core

# Testing the thing

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from fastai.tabular.all import * 

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

In [ ]:
df = pd.read_csv(path/'adult.csv')
df.head()

In [ ]:
len(df)

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [ ]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits)

In [ ]:
dls = to.dataloaders(bs=64)

In [ ]:
dls.show_batch()

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, 7e-3)

In [ ]:
learn.show_results()

In [ ]:
test_df = df.copy()
test_y = df['salary'].copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

In [ ]:
%%time
pred_y, _ = learn.get_preds(dl=dl)

Finding out where the model confidently predicted the wrong thing

In [ ]:
%%time
test_df['y_pred'] = pred_y.argmax(dim=1).clone().numpy()
test_df['y_pred_score'] = pred_y[range(len(pred_y)),pred_y.argmax(dim=1)].clone().numpy()

In [ ]:
test_df['y_true'] = test_y == '>=50k'
test_df['y_true'] = test_df['y_true'].astype(int)

In [ ]:
test_df.head()

In [ ]:
%%time
wrong = test_df['y_true'] != test_df['y_pred']

In [ ]:
for key, grp in test_df.loc[wrong].groupby('y_pred'):
    print(f'\npredicted class = {key:}')
    display(grp.sort_values('y_pred_score', ascending=False).head())

Finding:
- apparently the model is surprised by highly educated white males earning less than 50k

TODO: permutation importance https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html#sklearn.inspection.permutation_importance
TODO: partial dependency plot